
<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/1_Prepare_Data_Annotate_Images_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/1_Prepare_Data_Annotate_Images_Part1.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>



## Automatic Face Image Annotation  ##

We start off the data preparation phase by pre-processing images in our dataset for the unmasked category.

We have a selection of facial images and we need to draw bounding boxes around them. We could do that manually using a tool such as [labelImg](https://github.com/tzutalin/labelImg) however that would be slow, tedious and unnecessary for our purposes.

Instead we will use a facial detection algorithm (Haar cascade) to automatically detect faces and generate an accompanying XML file for each image with the coordinates of the face. Now this method is far from foolproof as it only works properly with full frontal images of faces but its a good starting point.



### Import required libraries

In [0]:
#!pip install lxml
#!pip install opencv-python
from lxml import etree as ET


import os
import cv2
import numpy as np
import glob
from pathlib import Path
import shutil

from PIL import Image, ImageFile

### Prepare dataset ###

#### Connect to Google Drive

In [2]:
from google.colab import drive

drive.mount('/content/drive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Copy dataset from Google Drive and extract


In [26]:
!mkdir -p /content/dataset/unmasked
%cd /content/drive/My Drive/Datasets/
%cp 1k_faces_00.zip /content/
!unzip 1k_faces_00.zip -d /content/dataset/unmasked

Archive:  1k_faces_00.zip
  inflating: /content/dataset/unmasked/0004STET6P.jpg  
  inflating: /content/dataset/unmasked/000N7AIAFT.jpg  
  inflating: /content/dataset/unmasked/00H858UYSD.jpg  
  inflating: /content/dataset/unmasked/00KPGHV40E.jpg  
  inflating: /content/dataset/unmasked/00P2LUXJW3.jpg  
  inflating: /content/dataset/unmasked/00PYA83V1P.jpg  
  inflating: /content/dataset/unmasked/00SD82OK2A.jpg  
  inflating: /content/dataset/unmasked/00TMYXA5DF.jpg  
  inflating: /content/dataset/unmasked/00V5CZZSSO.jpg  
  inflating: /content/dataset/unmasked/00W5NPIX4S.jpg  
  inflating: /content/dataset/unmasked/00XN46VW5G.jpg  
  inflating: /content/dataset/unmasked/01D47ZIGE0.jpg  
  inflating: /content/dataset/unmasked/01FCG0FF23.jpg  
  inflating: /content/dataset/unmasked/01KDGEL6IQ.jpg  
  inflating: /content/dataset/unmasked/01NH00JKMJ.jpg  
  inflating: /content/dataset/unmasked/01P734HDO0.jpg  
  inflating: /content/dataset/unmasked/02G2AQI22V.jpg  
  inflating: /content/

#### Set image directory ####

In [0]:
dataset_dir = "dataset/unmasked"

#### Download required dependency 

In [38]:
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/utils/lbpcascade_frontalface_improved.xml

--2020-06-04 12:32:14--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/utils/lbpcascade_frontalface_improved.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54039 (53K) [text/plain]
Saving to: ‘lbpcascade_frontalface_improved.xml’

lbpcascade_frontalf 100%[===================>]  52.77K  --.-KB/s    in 0.03s   

2020-06-04 12:32:14 (1.98 MB/s) - ‘lbpcascade_frontalface_improved.xml’ saved [54039/54039]



In [37]:
%cd /content/
path = os.getcwd()
filename = path + "/" + dataset_dir
face_cascade = cv2.CascadeClassifier('lbpcascade_frontalface_improved.xml')
dataset_labels_folder = dataset_dir + "-labels"
dataset_labels_path = os.getcwd()+ "/"+ dataset_labels_folder 
print(dataset_labels_path)
os.mkdir(dataset_labels_folder)


/content
/content/dataset/unmasked-labels


In [40]:
for subdir, dirs, files in os.walk(filename):
    for file in files:

        
        img_path=os.path.join(subdir, file)
        img_name=os.path.basename(img_path)

        if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):

            img = cv2.imread(img_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            h,w,bpp = np.shape(img)
            imgp = Image.open(img_path)
#            b, landmarks = detect_faces(imgp)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)


#            g=show_bboxes(imgp, b, landmarks)
            #print(len(faces))
            if len(faces) == 1 :
                try :

                    for bounding_boxes in faces:
                        face = img[int(bounding_boxes[1]):int(bounding_boxes[3]),
                        int(bounding_boxes[0]):int(bounding_boxes[2])]

                        
                        subdir_path, subdir_name = os.path.split(subdir)

                        root = ET.Element("annotation",verified="yes")
                        ET.SubElement(root, "folder").text=filename

                        ET.SubElement(root, "filename").text = img_name
                        ET.SubElement(root, "path").text = img_path

                        source=ET.SubElement(root, "source")
                        ET.SubElement(source, "database").text = "unknown"

                        size=ET.SubElement(root, "size")
                        ET.SubElement(size, "width").text = str(w)
                        ET.SubElement(size, "height").text = str(h)
                        ET.SubElement(size, "depth").text = str(bpp)

                        ET.SubElement(root, "segmented").text = "0"

                        obj=ET.SubElement(root, "object")
                        ET.SubElement(obj, "name").text = subdir_name
                        ET.SubElement(obj, "pose").text = "Unspecified"
                        ET.SubElement(obj, "truncated").text = "0"
                        ET.SubElement(obj, "difficult").text = "0"

                        box=ET.SubElement(obj, "bndbox")
                        ET.SubElement(box, "xmin").text = str(int(bounding_boxes[0]))
                        ET.SubElement(box, "ymin").text = str(int(bounding_boxes[1]))
                        ET.SubElement(box, "xmax").text = str(int(bounding_boxes[2])+int(bounding_boxes[1]))
                        ET.SubElement(box, "ymax").text = str(int(bounding_boxes[3]))

                        tree = ET.ElementTree(root)
                        tree.write(os.path.join(dataset_labels_path, os.path.splitext(img_name)[0] + '.xml'))
                        #tree.write(os.path.join(filename +"-labels", os.path.splitext(img_name)[0] + '.xml'))
                        
                except RuntimeError :
                    with open("delete.txt", "a") as myfile:
                        myfile.write(img_path+"\n")
                	#print(img_path)

                else :
                    with open("delete.txt", "a") as myfile:
                            myfile.write(img_path+"\n")
                    #print('no face')

error: ignored

### Save Dataset

In [45]:
source1 = filename + "/"
source2 = dataset_labels_path + "/"

destination = os.getcwd() + "/training_demo"+ "/images"
if not os.path.exists(destination):
    os.makedirs(destination)


def move_files(source,destination):
    files = os.listdir(source)
    for f in files:
        shutil.move(source+f, destination)


move_files(source1,destination)
move_files(source2,destination)

# deletes source directories, use as needed
os.rmdir(source2)
os.rmdir(source1)

Error: ignored

### Compress Dataset (Optional) ###

Follow this step if you wish to archive your dataset for future use

In [46]:
!zip -r unmasked-dataset.zip /content/training_demo

  adding: content/training_demo/ (stored 0%)
  adding: content/training_demo/images/ (stored 0%)
  adding: content/training_demo/images/VZGLQ059C1.jpg (deflated 0%)
  adding: content/training_demo/images/9V1Y7JYQNF.xml (deflated 47%)
  adding: content/training_demo/images/40RBMPI04G.jpg (deflated 0%)
  adding: content/training_demo/images/K9PFT0A194.xml (deflated 48%)
  adding: content/training_demo/images/0ZJLIECDHZ.jpg (deflated 0%)
  adding: content/training_demo/images/WAKUO7V02E.xml (deflated 47%)
  adding: content/training_demo/images/KA4FMGBO8H.xml (deflated 47%)
  adding: content/training_demo/images/99XOH68P4O.jpg (deflated 0%)
  adding: content/training_demo/images/9VFXN07J13.xml (deflated 48%)
  adding: content/training_demo/images/GMD7MLC7N3.jpg (deflated 0%)
  adding: content/training_demo/images/JPDKU084ZK.jpg (deflated 0%)
  adding: content/training_demo/images/J9KV0D4ZLL.xml (deflated 48%)
  adding: content/training_demo/images/11OTO6P279.jpg (deflated 1%)
  adding: con

In [0]:
!cp /content/unmasked-dataset.zip /content/gdrive/My\ Drive/Datasets/